In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

# Day 3: Multi-Stream Execution - Parallel Pipelines

---

## 🎯 The Hook: The Airport Terminal Analogy

**Ever noticed** how airports handle thousands of flights?

They don't use one runway for EVERYTHING - landing, takeoff, taxiing. They have:
- Multiple runways for parallel takeoffs/landings
- Multiple gates processing passengers simultaneously
- Baggage handling running independently

```
One-Runway Airport (Single Stream):
┌───────────────────────────────────────────────────────────────┐
│  [Flight 1 Land][Taxi][Unload] → [Flight 2 Land][Taxi]...     │
│                                                               │
│  Throughput: 12 flights/hour 😴                               │
└───────────────────────────────────────────────────────────────┘

Multi-Runway Airport (Multi-Stream):
┌───────────────────────────────────────────────────────────────┐
│  Runway 1: [Flight 1 Land] [Flight 4 Land] [Flight 7 Land]    │
│  Runway 2: [Flight 2 Land] [Flight 5 Land] [Flight 8 Land]    │
│  Runway 3: [Flight 3 Land] [Flight 6 Land] [Flight 9 Land]    │
│                                                               │
│  Throughput: 36 flights/hour! 🚀                              │
└───────────────────────────────────────────────────────────────┘
```

**Your GPU has many SMs** - small kernels might only use a fraction. Fill the rest with MORE kernels!

---

## 📚 Learning Objectives

By the end of this lesson, you will be able to:
1. **Launch concurrent kernels** in multiple streams
2. **Understand** when kernels can run simultaneously (resource limits)
3. **Use stream priorities** to ensure critical work completes first
4. **Design** multi-stream pipelines for maximum throughput
5. **Balance** the number of streams for optimal performance

---

## 🃏 Concept Card: Multiple Parallel Pipelines

```
╔══════════════════════════════════════════════════════════════════╗
║  🚀 MULTI-STREAM = MULTIPLE PARALLEL PIPELINES                   ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  ANALOGY: Grocery Store Checkout Lines                           ║
║                                                                  ║
║  One Line (Single Stream):         Multiple Lines (Multi-Stream):║
║  ┌────────────────────────┐        ┌────────────────────────┐    ║
║  │ 🧑‍🤝‍🧑🧑‍🤝‍🧑🧑‍🤝‍🧑🧑‍🤝‍🧑🧑‍🤝‍🧑🧑‍🤝‍🧑 → [💳]   │        │ 🧑‍🤝‍🧑🧑‍🤝‍🧑 → [💳] Line 1 │    ║
║  │   Everyone waits!      │        │ 🧑‍🤝‍🧑🧑‍🤝‍🧑 → [💳] Line 2 │    ║
║  └────────────────────────┘        │ 🧑‍🤝‍🧑🧑‍🤝‍🧑 → [💳] Line 3 │    ║
║                                    └────────────────────────┘    ║
║                                                                  ║
║  GPU Multi-Stream Execution:                                     ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │         SM0    SM1    SM2    SM3    SM4    SM5             │  ║
║  │  Stream0: ████   ████                                      │  ║
║  │  Stream1:              ████   ████                         │  ║
║  │  Stream2:                          ████   ████             │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║     Each stream gets a slice of the GPU - all run at once!       ║
║                                                                  ║
║  💡 KEY INSIGHT: Small kernels leave GPU resources unused.       ║
║     Fill those resources with concurrent work from other streams!║
║                                                                  ║
║  WHEN IT HELPS:                                                  ║
║  ✓ Kernels don't fully occupy GPU                                ║
║  ✓ Independent workloads available                               ║
║  ✓ Mix of small and large kernels                                ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
```

---

## Part 1: Concurrent Kernel Execution

### When Can Kernels Run Concurrently?

```
Requirements for Concurrent Kernels:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
1. Different streams
2. Enough GPU resources (SMs, registers, shared memory)
3. No dependencies between kernels
4. Device supports concurrent kernels (check capability)

┌─────────────────────────────────────────────────────────┐
│                      GPU SMs                            │
├─────────────┬─────────────┬─────────────┬──────────────┤
│    SM 0     │    SM 1     │    SM 2     │    SM 3      │
├─────────────┼─────────────┼─────────────┼──────────────┤
│ Kernel A    │ Kernel A    │ Kernel B    │ Kernel B     │
│ blocks      │ blocks      │ blocks      │ blocks       │
└─────────────┴─────────────┴─────────────┴──────────────┘
         ↑ Multiple kernels share the GPU!
```

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates running multiple kernels concurrently by placing them in different streams, and compares sequential vs concurrent execution times.

In [ ]:
%%writefile concurrent_kernels.cu
// concurrent_kernels.cu - Running multiple kernels at once
#include <stdio.h>
#include <cuda_runtime.h>

// Small kernel - uses few resources
__global__ void smallKernel(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        for (int i = 0; i < 1000; i++) {
            data[tid] = sinf(data[tid]);
        }
    }
}

int main() {
    const int N = 1 << 16;  // Small size so kernels fit together
    const int NUM_STREAMS = 4;
    
    // Check concurrent kernel support
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Concurrent kernels: %s\n", 
           prop.concurrentKernels ? "Yes" : "No");
    printf("Max concurrent kernels: ~%d (estimate)\n",
           prop.multiProcessorCount);  // Rough estimate
    
    // Allocate
    float* d_data[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaMalloc(&d_data[i], N * sizeof(float));
    }
    
    // Create streams
    cudaStream_t streams[NUM_STREAMS];
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&streams[i]);
    }
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // ============================================
    // Sequential Execution (single stream)
    // ============================================
    cudaEventRecord(start);
    
    for (int i = 0; i < NUM_STREAMS; i++) {
        // All in default stream
        smallKernel<<<64, 256>>>(d_data[i], N);
    }
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float seqTime;
    cudaEventElapsedTime(&seqTime, start, stop);
    
    // ============================================
    // Concurrent Execution (multiple streams)
    // ============================================
    cudaEventRecord(start);
    
    for (int i = 0; i < NUM_STREAMS; i++) {
        // Each in its own stream - can run concurrently!
        smallKernel<<<64, 256, 0, streams[i]>>>(d_data[i], N);
    }
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float concTime;
    cudaEventElapsedTime(&concTime, start, stop);
    
    printf("\nResults:\n");
    printf("Sequential: %.2f ms\n", seqTime);
    printf("Concurrent: %.2f ms\n", concTime);
    printf("Speedup:    %.2fx\n", seqTime / concTime);
    
    // Cleanup
    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(streams[i]);
        cudaFree(d_data[i]);
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o concurrent_kernels concurrent_kernels.cu
!./concurrent_kernels

---

## Part 2: Stream Priorities

### 🔷 CUDA C++ Implementation (Primary)

This example demonstrates how to create prioritized streams where high-priority work gets GPU resources first. Note: LOWER number = HIGHER priority!

### Priority Use Cases

```
When to Use Stream Priorities:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

1. Interactive Applications
   └─ High: UI rendering, user input processing
   └─ Low: Background computation, prefetching

2. Real-time Systems  
   └─ High: Critical path operations
   └─ Low: Optional/deferrable work

3. Multi-tenant GPU
   └─ High: Latency-sensitive workloads
   └─ Low: Throughput-oriented batch jobs

Limitations:
• Priority only affects scheduling at SM level
• Doesn't preempt running kernels
• Effect depends on resource availability
```

In [ ]:
%%writefile stream_priorities.cu
// stream_priorities.cu - Prioritizing work
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void work(float* data, int n, const char* name) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        for (int i = 0; i < 1000; i++) {
            data[tid] = sqrtf(data[tid] + 1.0f);
        }
    }
}

int main() {
    // ============================================
    // Query Priority Range
    // ============================================
    int lowPriority, highPriority;
    cudaDeviceGetStreamPriorityRange(&lowPriority, &highPriority);
    
    printf("Priority range: %d (low/default) to %d (high)\n",
           lowPriority, highPriority);
    // Note: LOWER number = HIGHER priority!
    
    // ============================================
    // Create Prioritized Streams
    // ============================================
    cudaStream_t highPriorityStream, lowPriorityStream;
    
    cudaStreamCreateWithPriority(&highPriorityStream, 
                                  cudaStreamNonBlocking, 
                                  highPriority);
    
    cudaStreamCreateWithPriority(&lowPriorityStream, 
                                  cudaStreamNonBlocking, 
                                  lowPriority);
    
    const int N = 1 << 16;
    float *d_high, *d_low;
    cudaMalloc(&d_high, N * sizeof(float));
    cudaMalloc(&d_low, N * sizeof(float));
    
    // Launch many low-priority kernels
    for (int i = 0; i < 10; i++) {
        work<<<64, 256, 0, lowPriorityStream>>>(d_low, N, "LOW");
    }
    
    // Launch high-priority kernel (should get resources first)
    work<<<64, 256, 0, highPriorityStream>>>(d_high, N, "HIGH");
    
    cudaDeviceSynchronize();
    
    printf("High priority work completed!\n");
    
    cudaStreamDestroy(highPriorityStream);
    cudaStreamDestroy(lowPriorityStream);
    cudaFree(d_high);
    cudaFree(d_low);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o stream_priorities stream_priorities.cu
!./stream_priorities

---

## Part 3: Designing Multi-Stream Applications

### 🔷 CUDA C++ Implementation (Primary)

**Pattern: Producer-Consumer Streams**

This example demonstrates a producer-consumer pattern where:
- A producer stream generates data chunks
- Events mark when each chunk is ready  
- A consumer stream waits for each event before processing

In [ ]:
%%writefile producer_consumer.cu
// producer_consumer.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void produce(float* data, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        data[tid] = (float)tid;  // Generate data
    }
}

__global__ void consume(float* data, float* result, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        result[tid] = data[tid] * 2.0f;  // Process data
    }
}

int main() {
    const int N = 1 << 20;
    const int NUM_CHUNKS = 4;
    const int CHUNK = N / NUM_CHUNKS;
    
    float *d_data, *d_result;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_result, N * sizeof(float));
    
    cudaStream_t producerStream, consumerStream;
    cudaStreamCreate(&producerStream);
    cudaStreamCreate(&consumerStream);
    
    cudaEvent_t chunkReady[NUM_CHUNKS];
    for (int i = 0; i < NUM_CHUNKS; i++) {
        cudaEventCreate(&chunkReady[i]);
    }
    
    // Pipeline: Producer creates data, consumer processes it
    for (int i = 0; i < NUM_CHUNKS; i++) {
        // Producer generates chunk i
        produce<<<(CHUNK+255)/256, 256, 0, producerStream>>>(
            d_data + i*CHUNK, CHUNK);
        
        // Mark chunk as ready
        cudaEventRecord(chunkReady[i], producerStream);
        
        // Consumer waits for chunk, then processes
        cudaStreamWaitEvent(consumerStream, chunkReady[i]);
        consume<<<(CHUNK+255)/256, 256, 0, consumerStream>>>(
            d_data + i*CHUNK, d_result + i*CHUNK, CHUNK);
    }
    
    cudaDeviceSynchronize();
    printf("Producer-consumer pipeline complete!\n");
    
    // Cleanup
    for (int i = 0; i < NUM_CHUNKS; i++) {
        cudaEventDestroy(chunkReady[i]);
    }
    cudaStreamDestroy(producerStream);
    cudaStreamDestroy(consumerStream);
    cudaFree(d_data);
    cudaFree(d_result);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o producer_consumer producer_consumer.cu
!./producer_consumer

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Python/Numba Multi-Stream Demo (OPTIONAL)

@cuda.jit
def kernel_a(data):
    tid = cuda.grid(1)
    if tid < data.shape[0]:
        for _ in range(500):
            data[tid] = data[tid] * 1.0001

@cuda.jit
def kernel_b(data):
    tid = cuda.grid(1)
    if tid < data.shape[0]:
        for _ in range(500):
            data[tid] = data[tid] + 0.0001

n = 1 << 18
num_streams = 4

# Create streams and data
streams = [cuda.stream() for _ in range(num_streams)]
d_arrays = [cuda.device_array(n, dtype=np.float32) for _ in range(num_streams)]

# Sequential (baseline)
start = time.time()
for i in range(num_streams):
    kernel_a[(n+255)//256, 256](d_arrays[i])
cuda.synchronize()
seq_time = time.time() - start

# Concurrent (multi-stream)
start = time.time()
for i in range(num_streams):
    kernel_a[(n+255)//256, 256, streams[i]](d_arrays[i])
for s in streams:
    s.synchronize()
conc_time = time.time() - start

print(f"Sequential: {seq_time*1000:.2f} ms")
print(f"Concurrent: {conc_time*1000:.2f} ms")
print(f"Speedup: {seq_time/conc_time:.2f}x")

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile multistream_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
            exit(1); \
        } \
    } while(0)

// Small kernel for concurrency testing
__global__ void smallKernel(float* data, int n, int iters) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = data[idx];
        for (int i = 0; i < iters; i++) {
            val = sinf(val) + 0.1f;
        }
        data[idx] = val;
    }
}

// Longer kernel
__global__ void longKernel(float* data, int n, int iters) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = data[idx];
        for (int i = 0; i < iters; i++) {
            val = sinf(val) * cosf(val) + 0.001f;
        }
        data[idx] = val;
    }
}

// ============================================================
// Exercise 1: Find Concurrency Limit
// ============================================================

void exercise1_concurrencyLimit() {
    printf("=== Exercise 1: Find Concurrency Limit ===\n");
    
    const int n = 1024;  // Small work per kernel
    const int iters = 100;
    
    float* d_data;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    int testCounts[] = {1, 2, 4, 8, 16, 32, 64, 128};
    int numTests = sizeof(testCounts) / sizeof(testCounts[0]);
    
    printf("%-15s %-15s %-15s\n", "Num Kernels", "Total Time", "Time/Kernel");
    printf("----------------------------------------------\n");
    
    float singleKernelTime = 0;
    
    for (int t = 0; t < numTests; t++) {
        int numKernels = testCounts[t];
        
        cudaStream_t* streams = (cudaStream_t*)malloc(numKernels * sizeof(cudaStream_t));
        for (int i = 0; i < numKernels; i++) {
            cudaStreamCreate(&streams[i]);
        }
        
        // Warmup
        for (int i = 0; i < numKernels; i++) {
            smallKernel<<<1, 256, 0, streams[i]>>>(d_data, n, iters);
        }
        cudaDeviceSynchronize();
        
        cudaEventRecord(start);
        for (int i = 0; i < numKernels; i++) {
            smallKernel<<<1, 256, 0, streams[i]>>>(d_data, n, iters);
        }
        cudaDeviceSynchronize();
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        
        float ms;
        cudaEventElapsedTime(&ms, start, stop);
        
        if (numKernels == 1) singleKernelTime = ms;
        
        float idealTime = singleKernelTime;  // Fully concurrent
        float efficiency = idealTime / ms * 100.0f;
        
        printf("%-15d %-12.3f ms %-12.3f ms (%.0f%% efficient)\n", 
               numKernels, ms, ms / numKernels, efficiency);
        
        for (int i = 0; i < numKernels; i++) {
            cudaStreamDestroy(streams[i]);
        }
        free(streams);
    }
    printf("\n");
    
    cudaFree(d_data);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

// ============================================================
// Exercise 2: Priority Scheduling
// ============================================================

void exercise2_priorityScheduling() {
    printf("=== Exercise 2: Priority Scheduling ===\n");
    
    const int n = 1 << 16;
    const int longIters = 1000;
    const int shortIters = 50;
    
    int leastPriority, greatestPriority;
    cudaDeviceGetStreamPriorityRange(&leastPriority, &greatestPriority);
    printf("Priority range: %d (least) to %d (greatest)\n", leastPriority, greatestPriority);
    
    if (leastPriority == greatestPriority) {
        printf("Stream priorities not supported on this device!\n\n");
        return;
    }
    
    float *d_data1, *d_data2;
    CHECK_CUDA(cudaMalloc(&d_data1, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_data2, n * sizeof(float)));
    
    cudaStream_t lowPriorityStream, highPriorityStream;
    cudaStreamCreateWithPriority(&lowPriorityStream, cudaStreamNonBlocking, leastPriority);
    cudaStreamCreateWithPriority(&highPriorityStream, cudaStreamNonBlocking, greatestPriority);
    
    cudaEvent_t startLow, stopLow, startHigh, stopHigh;
    cudaEventCreate(&startLow);
    cudaEventCreate(&stopLow);
    cudaEventCreate(&startHigh);
    cudaEventCreate(&stopHigh);
    
    int grid = (n + 255) / 256;
    
    // Launch long-running low-priority kernel
    cudaEventRecord(startLow, lowPriorityStream);
    for (int i = 0; i < 10; i++) {
        longKernel<<<grid, 256, 0, lowPriorityStream>>>(d_data1, n, longIters);
    }
    cudaEventRecord(stopLow, lowPriorityStream);
    
    // Small delay, then launch high-priority kernel
    cudaEventRecord(startHigh, highPriorityStream);
    smallKernel<<<grid, 256, 0, highPriorityStream>>>(d_data2, n, shortIters);
    cudaEventRecord(stopHigh, highPriorityStream);
    
    cudaDeviceSynchronize();
    
    float lowMs, highMs;
    cudaEventElapsedTime(&lowMs, startLow, stopLow);
    cudaEventElapsedTime(&highMs, startHigh, stopHigh);
    
    printf("Low priority work:  %.2f ms\n", lowMs);
    printf("High priority work: %.2f ms\n", highMs);
    printf("High priority completed while low priority was running!\n\n");
    
    cudaStreamDestroy(lowPriorityStream);
    cudaStreamDestroy(highPriorityStream);
    cudaEventDestroy(startLow);
    cudaEventDestroy(stopLow);
    cudaEventDestroy(startHigh);
    cudaEventDestroy(stopHigh);
    cudaFree(d_data1);
    cudaFree(d_data2);
}

// ============================================================
// Exercise 3: Multi-Stage Pipeline
// ============================================================

__global__ void stage1_preprocess(const float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = fabsf(input[idx]) + 1.0f;  // Ensure positive
    }
}

__global__ void stage2_compute(const float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = input[idx];
        for (int i = 0; i < 100; i++) {
            val = sqrtf(val) + logf(val + 1.0f);
        }
        output[idx] = val;
    }
}

__global__ void stage3_postprocess(const float* input, float* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        output[idx] = tanhf(input[idx]);  // Normalize to [-1, 1]
    }
}

void exercise3_multiStagePipeline() {
    printf("=== Exercise 3: Multi-Stage Pipeline ===\n");
    
    const int n = 1 << 20;  // 1M elements
    const int numChunks = 4;
    const int chunkSize = n / numChunks;
    
    // Allocate buffers for pipeline stages
    float *d_input, *d_stage1, *d_stage2, *d_output;
    CHECK_CUDA(cudaMalloc(&d_input, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_stage1, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_stage2, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_output, n * sizeof(float)));
    
    // Create streams for each pipeline stage
    cudaStream_t streams[3];
    for (int i = 0; i < 3; i++) {
        cudaStreamCreate(&streams[i]);
    }
    
    // Events for synchronization between stages
    cudaEvent_t stage1Done[numChunks], stage2Done[numChunks];
    for (int i = 0; i < numChunks; i++) {
        cudaEventCreate(&stage1Done[i]);
        cudaEventCreate(&stage2Done[i]);
    }
    
    int grid = (chunkSize + 255) / 256;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    
    for (int chunk = 0; chunk < numChunks; chunk++) {
        int offset = chunk * chunkSize;
        
        // Stage 1: Preprocess
        if (chunk > 0) {
            // Wait for previous stage1 to complete before reusing stage1 buffer slot
            cudaStreamWaitEvent(streams[0], stage1Done[(chunk - 1) % 2]);
        }
        stage1_preprocess<<<grid, 256, 0, streams[0]>>>(
            d_input + offset, d_stage1 + offset, chunkSize);
        cudaEventRecord(stage1Done[chunk % 2], streams[0]);
        
        // Stage 2: Main computation (waits for stage 1)
        cudaStreamWaitEvent(streams[1], stage1Done[chunk % 2]);
        stage2_compute<<<grid, 256, 0, streams[1]>>>(
            d_stage1 + offset, d_stage2 + offset, chunkSize);
        cudaEventRecord(stage2Done[chunk % 2], streams[1]);
        
        // Stage 3: Postprocess (waits for stage 2)
        cudaStreamWaitEvent(streams[2], stage2Done[chunk % 2]);
        stage3_postprocess<<<grid, 256, 0, streams[2]>>>(
            d_stage2 + offset, d_output + offset, chunkSize);
    }
    
    cudaDeviceSynchronize();
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    printf("3-stage pipeline with %d chunks: %.2f ms\n", numChunks, ms);
    printf("Stages: preprocess -> compute -> postprocess\n\n");
    
    for (int i = 0; i < 3; i++) cudaStreamDestroy(streams[i]);
    for (int i = 0; i < numChunks; i++) {
        cudaEventDestroy(stage1Done[i]);
        cudaEventDestroy(stage2Done[i]);
    }
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_input);
    cudaFree(d_stage1);
    cudaFree(d_stage2);
    cudaFree(d_output);
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║           Multi-Stream Exercises                             ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Device: %s\n", prop.name);
    printf("Concurrent Kernels: %s\n", prop.concurrentKernels ? "Yes" : "No");
    printf("Max Threads/SM: %d\n\n", prop.maxThreadsPerMultiProcessor);
    
    exercise1_concurrencyLimit();
    exercise2_priorityScheduling();
    exercise3_multiStagePipeline();
    
    printf("══════════════════════════════════════════════════════════════\n");
    printf("                    All exercises completed!\n");
    printf("══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o multistream_exercises multistream_exercises.cu && ./multistream_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Find Concurrency Limit
Experimentally determine how many small kernels can run concurrently on your GPU.

### Exercise 2: Priority Scheduling
Create a scenario where high-priority work completes noticeably faster.

### Exercise 3: Multi-Stage Pipeline
```cpp
// Implement a 3-stage pipeline:
// Stage 1: Load and preprocess
// Stage 2: Main computation
// Stage 3: Post-process and store
```

---

## 🎯 Key Takeaways

```
╔══════════════════════════════════════════════════════════════════╗
║            MULTI-STREAM EXECUTION MASTERY                        ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  🚀 Core Concept: PARALLEL PIPELINES                             ║
║     Multiple checkout lines - everyone gets served faster!       ║
║                                                                  ║
║  📋 Essential API:                                               ║
║  ┌────────────────────────────────────────────────────────────┐  ║
║  │  cudaDeviceGetStreamPriorityRange(&low, &high)             │  ║
║  │  cudaStreamCreateWithPriority(&s, flags, priority)         │  ║
║  │  // Lower number = higher priority!                        │  ║
║  └────────────────────────────────────────────────────────────┘  ║
║                                                                  ║
║  🔢 Concurrency Limits:                                          ║
║     • Max concurrent kernels: typically 16-128                   ║
║     • Limited by: SMs, registers, shared memory                  ║
║     • Check: deviceQuery for your GPU's limits                   ║
║                                                                  ║
║  ⚠️  When Multi-Stream DOESN'T Help:                             ║
║     • Kernel already fills GPU (no room for more)               ║
║     • Memory bandwidth saturated                                 ║
║     • Operations have dependencies                               ║
║                                                                  ║
║  📊 Design Patterns:                                             ║
║     Producer-Consumer: Stream A produces, Stream B consumes      ║
║     Pipeline Stages:   Stage 1 → Stage 2 → Stage 3 in parallel   ║
║     Work Splitting:    Divide large task across streams          ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
```

---

## 🔮 What's Next?

**Day 4: Events & Synchronization** - Fine-grained control!

Streams work great for independent work, but what about dependencies? Tomorrow you'll learn:

```
Today:                    Tomorrow (Day 4):
Independent streams       Streams with dependencies

[Stream 1: ████████]      [Stream 1: ████]─event─┐
[Stream 2: ████████]      [Stream 2: wait for event]→[████]
                                    ↑
                          "Don't start until Stream 1 reaches here!"
```

Events are your stopwatches AND synchronization checkpoints! ⏱️